In [48]:
#To authenticate the user to make more requests at a time
import json
import numpy as np
import pandas as pd
from getpass import getpass, getuser

import re
import string
import csv

import requests
from requests.auth import HTTPBasicAuth

# credentials = json.loads(open('Sec\credentials.json').read())
user=input("Enter your github Username")
passw=getpass()
authentication = HTTPBasicAuth(user, passw)
#We authorize our requests using this authentication variable (equivalen to auth=(username,password) in a request)


tot_repos_fetched=0
df=pd.read_csv("users.csv")

repos_data=[]# This is where we will store all our repository data 

for i in df['Users']:
    data=requests.get('https://api.github.com/users/' + i, auth = authentication)
    data = data.json()
    pretty_data=json.dumps(data,indent=4)
    # print(pretty_data)
    user_name=data['name']
    user_location=data['location']
    user_email=data['email']
    user_bio=data['bio']
    user_public_repos_num=data['public_repos']
    user_repos_url=data['repos_url'] #provides the github API to fetch the repository details for the user
    
    

    page_no=1
    #Since the given api only fetches about 30 repositories we use a seperatre parameter known as page numbers to fetch the further repos
    while (True):
        response = requests.get(user_repos_url, auth = authentication)
        response = response.json()
        repos_data = repos_data + response
        repos_fetched = len(response)
        tot_repos_fetched+=repos_fetched
        print("repositories fetched: {}".format(repos_fetched))
        if (repos_fetched == 30):
            page_no = page_no + 1
            user_repos_url = data['repos_url'] + '?page=' + str(page_no)
        else:
            break
print("Total repositroy fetched for all users: "+str(tot_repos_fetched))


for i in range(tot_repos_fetched):
    repos_data[i]['name']=" ".join(''.join(i for i in list(repos_data[i]['name']) if i in string.ascii_lowercase or i in string.ascii_uppercase or i==' ').split())
    try:
        repos_data[i]['description']=" ".join(''.join(i for i in list(repos_data[i]['description']) if i in string.ascii_lowercase or i in string.ascii_uppercase or i==' ').split())
    except:
        pass

filename="repos_all.csv"
fields=["repo_name","repo_description"]
# if str(repo_description)=='none':
#     repo_description=""
rows=[[repos_data[i]["name"],repos_data[i]["description"]] for i in range(tot_repos_fetched)]

with open(filename,'w')as csvfile:
    csvwriter = csv.writer(csvfile)
   
    # writing the fields
    csvwriter.writerow(fields)
     
    # writing the data rows
    csvwriter.writerows(rows)

repositories fetched: 10
repositories fetched: 21
repositories fetched: 30
repositories fetched: 10
Total repositroy fetched for all users: 71
CalHousePredictor
